# Computergestützte Mathematik zur Analysis

31.01.2019

&copy; Rüdiger W. Braun

Bitte lesen Sie die Hinweise zur [Klausurdurchführung](https://github.com/Ruediger-Braun/compana18#prufungsdurchfuhrung)

In [ ]:
import numpy as np
from sympy import *
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay 
from mayavi import mlab
init_printing()
mlab.init_notebook()

# benutzerspezifische trigonometrische Vereinfachungen

Aufgabe:  ersetze $\sin(x) + \sin(y)$ durch
$2 \sin{\left (\frac{x}{2} + \frac{y}{2} \right )} \cos{\left (\frac{x}{2} - \frac{y}{2} \right )}$

In [ ]:
x = Symbol('x')
y = Symbol('y')
b = sin(x) + sin(y)
b

In [ ]:
b.count_ops(visual=True)

In [ ]:
def my_measure(expr):
    opc = expr.count_ops(visual=True)
    print(opc)   # zur Fehlersuche
    wert = {}
    wert['ADD'] = 1
    wert['SIN'] = 1
    wert['MUL'] = -100  # Multiplikationen werden belohnt
    return opc.subs(wert)

In [ ]:
my_measure(b)

In [ ]:
b.trigsimp(method='fu', measure=my_measure)  
# TypeError, weil nicht genügend Werte vereinbart sind

In [ ]:
def my_measure(expr):
    opc = expr.count_ops(visual=True)
    wert = {}
    wert['ADD'] = 1
    wert['SIN'] = 1
    wert['MUL'] = -100
    wert['COS'] = 1
    wert['DIV'] = 1 
    wert['SUB'] = 1
    return opc.subs(wert)

In [ ]:
b.trigsimp(method='fu', measure=my_measure)

In [ ]:
_.trigsimp()

In [ ]:
e = tan(x) + cot(y)
e.simplify()

In [ ]:
def my_measure(expr):
    opc = expr.count_ops(visual=True)
    wert = {}
    wert['ADD'] = 1
    wert['SIN'] = 1
    wert['COS'] = 1
    wert['TAN'] = 100
    wert['COT'] = 100
    wert['DIV'] = 1
    return opc.subs(wert)

In [ ]:
e1 = e.trigsimp(method='fu', measure=my_measure)
e1

In [ ]:
e2 = e1.ratsimp()
e2

In [ ]:
numer(e2).trigsimp() / denom(e2)

# Mayavi

In [ ]:
rn = np.linspace(0, 5*np.pi)
theta_n = np.linspace(0, 2*np.pi)
R, Theta = np.meshgrid(rn, theta_n)
X = R*np.cos(Theta)
Y = R*np.sin(Theta)
W = np.cos(np.sqrt(X**2+Y**2))

In [ ]:
mlab.mesh(X, Y, W)

In [ ]:
mlab.close()
mlab.mesh(X, Y, W, scalars=1/(R+10), colormap='inferno')

In [ ]:
def moebiusband():
    t = Symbol('t')  # Winkel theta gegen die Ebene
    s = Symbol('s')  # Seele
    M = Matrix([ 3*cos(t) + s*sin(t/2),
                       3*sin(t),
                       s*cos(t/2)])
    sn = np.linspace(-1, 1)
    tn = np.linspace(0, 2*np.pi)
    S1, T1 = np.meshgrid(sn, tn)
    Sn, Tn = S1.T, T1.T
    fn = [lambdify((s,t), M[j], 'numpy') for j in range(3)]
    Xn = [fn[j](Sn, Tn) for j in range(3)]
    return M, Xn, T1 

In [ ]:
def normale(M):
    s = Symbol('s')
    t = Symbol('t')
    seele = M.subs(s, 0)
    Mt = seele.diff(t)
    Ms = M.diff(s)
    display(Mt, Ms)
    N = Mt.cross(Ms)
    display(N)
    N_ein = seele + s*N/N.norm()
    sn = np.linspace(0.05, 1)
    tn = np.linspace(0, 2*np.pi)
    S1, T1 = np.meshgrid(sn, tn)
    Sn, Tn = S1.T, T1.T
    Nn = [lambdify((s,t), N_ein[j], 'numpy') for j in range(3)]
    Xn = [Nn[j](Sn, Tn) for j in range(3)]
    return Xn

In [ ]:
M, Xn, T1 = moebiusband()
Nn = normale(M)

In [ ]:
mlab.close()
mlab.mesh(Xn[0], Xn[1], Xn[2], scalars=T1)

In [ ]:
mlab.mesh(Nn[0], Nn[1], Nn[2], color=(0, .3, 0)) 

In [ ]:
mlab.close()
x = [0, 1, 0, 0]
y = [0, 0, 0, 1]
z = [0, 0, 1, 0]
dreiecke = [[0,1,2], [0,1,3], [0,2,3],[1,2,3]]
mlab.triangular_mesh(x, y, z, dreiecke, colormap='viridis')

In [ ]:
mlab.close()
mlab.triangular_mesh(x, y, z, dreiecke, scalars=np.arange(4), colormap='hot')
# legt die Farben der Ecken fest

Triangulierungen

In [ ]:
from scipy.spatial import Delaunay

In [ ]:
r = np.linspace(1,3,10)
theta = np.linspace(0, 2*np.pi, 20)
R, Theta = np.meshgrid(r, theta)

In [ ]:
X = R*np.cos(Theta)
Y = R*np.sin(Theta)
Xf = X.flatten()
Yf = Y.flatten()

In [ ]:
punkte = np.vstack([Xf, Yf]).T
dreiecke = Delaunay(punkte);

In [ ]:
d = dreiecke.simplices[0]
d

In [ ]:
plt.triplot(Xf, Yf, dreiecke.simplices)
plt.axis('equal');

Delaunay trianguliert immer die konvexe Hülle

In [ ]:
z = Symbol('z')

In [ ]:
v = z/(1+z**4)
pole = [complex(p.n()) for p in solveset(Eq(denom(v), 0))]

In [ ]:
v

In [ ]:
pole

In [ ]:
epsilon = .07
r = np.linspace(0, 2, 100)
phi = np.linspace(0, 2*np.pi, 400)
R, Phi = np.meshgrid(r, phi)
Rf = R.flatten()
Phi_f = Phi.flatten()
Z = Rf*np.exp(1j*Phi_f)

tn = np.linspace(0, 2*np.pi, 120) # kleine Kreise um die Pole
for pol in pole:
    z_neu = pol + epsilon*np.exp(1j*tn)
    Z = np.hstack([Z, z_neu])
X = Z.real
Y = Z.imag
V = Z / (1+Z**4)
W = abs(V)
Theta = np.angle(V)
d = Delaunay(np.array([X, Y]).T)

In [ ]:
plt.figure(figsize=(4,4))
plt.triplot(X, Y, d.simplices)
plt.axis(xmin=.6, xmax=.8, ymin=.6, ymax=.8);

In [ ]:
def dreieck_gueltig(simplex, d):
    punkte = [d.points[nr] for nr in simplex]
    for p in punkte:
        z = p[0] + 1j*p[1]
        for pol in pole:
            if abs(pol-z) < .999*epsilon:
                return False
    return True

simplices = [s for s in d.simplices if dreieck_gueltig(s, d)]
plt.figure(figsize=(4,4))
plt.triplot(X, Y, simplices)
plt.axis(xmin=.6, xmax=.8, ymin=.6, ymax=.8);

In [ ]:
mlab.close()
mlab.triangular_mesh(X, Y, W, simplices, 
                         scalars=Theta, colormap='hsv')

Ein Beispiel aus Lektion 5

In [ ]:
xn = np.linspace(-1.8, 1.2)
yn = np.linspace(-1.3, 1.3)
zn = np.linspace(-1.3, 1.3)
X, Y, Z = np.meshgrid(xn, yn, zn)
H = Y**2 - (X-1)*(X+1)**2 - Z**2
mlab.close()
mlab.contour3d(H, contours=[0])